<a href="https://colab.research.google.com/github/ShlokArora2709/ESG-Project/blob/main/Model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import re
import string
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

In [2]:
df1=pd.read_csv('/content/drive/MyDrive/ESG_Data/ESG_w_cont.csv')
df2=pd.read_csv('/content/drive/MyDrive/ESG_Data/ESG_w_cont2.csv')
df3=pd.read_csv('/content/drive/MyDrive/ESG_Data/ESG_w_cont3.1.csv')
df4=pd.read_csv('/content/drive/MyDrive/ESG_Data/ESG_w_cont4.csv')

In [3]:
df=pd.concat([df1,df2,df3,df4])

In [4]:
df

,Unnamed: 0,Company Name,ESG Rating,E,S,G
0,0,AAC TECHNOLOGIES HOLDINGS INC.,BBB,['瑞声科技 搜索 中文简体 English 无数据 辰瑞光学 RichTap 无数据 首页...,['瑞声科技 搜索 中文简体 English 无数据 辰瑞光学 RichTap 无数据 首页...,['瑞声科技 搜索 中文简体 English 无数据 辰瑞光学 RichTap 无数据 首页...
1,1,APPLE INC.,BBB,['Environment - Apple Apple Store Mac iPad iPh...,"['403 Forbidden 403 Forbidden nginx...', 'Yaho...",['Ethics and Compliance - Apple Apple Store Ma...
2,2,EQT AB,AA,['Just a moment... Enable JavaScript and cooki...,['Just a moment... Enable JavaScript and cooki...,['EQT Policies & Statements English Investors ...
3,3,Addtech AB,AA,"[""Environment - English This site is optimized...",['Sustainability for Addtech - English This si...,['Sustainability for Addtech - English This si...
4,4,Skanska AB,A,"[""Sustainability | www.usa.skanska.com Skip to...",['ESG | www.skanska.pl Skip to content Menu Me...,['ESG | www.skanska.pl Skip to content Menu Me...
...,...,...,...,...,...,...
94,395,IDEX CORPORATION,AA,['Sustainability at IDEX | IDEX Corporation / ...,"['IDEX Security Rating, Vendor Risk Report, an...",['Compliance & Integrity - IDEX Corporation GA...
95,396,VODAFONE IDEA LIMITED,B,"[""Tackling carbon emissions Close Country Sele...",['Vodafone Idea Unmasks SSL/TLS Security Threa...,"[""Our principles and policies Close Country Se..."
96,397,IDFC FIRST BANK LIMITED,A,['Scaling up Sustainability efforts | IDFC FIR...,['FIRST IMPACT is IDFC FIRST Bank’s Corporate ...,['Ethical Banking: Why the World Needs it Righ...
97,398,"IEIT SYSTEMS Co., Ltd,",A,['A Determined Practitioner of ESG! IEIT SYSTE...,['Legal Notice— IEITSYSTEMS The site uses Cook...,['A Determined Practitioner of ESG! IEIT SYSTE...


In [5]:
df.columns

Index(['Unnamed: 0', 'Company Name', 'ESG Rating', 'E', 'S', 'G'], dtype='object')

In [6]:
df.isnull().sum()

,0
Unnamed: 0,0
Company Name,0
ESG Rating,0
E,0
S,0
G,0


In [7]:
df.iloc[22]["G"]

'[\'Aero Engine Corporation of China - Wikipedia Jump to content Main menu Main menu move to sidebar hide Navigation Main page Contents Current events Random article About Wikipedia Contact us Contribute Help Learn to edit Community portal Recent changes Upload file Search Search Donate Appearance Create account Log in Personal tools Create account Log in Pages for logged out editors learn more Contributions Talk CentralNotice Contents move to sidebar hide (Top) 1 U.S. sanctions 2 See also 3 References 4 External links Toggle the table of contents Aero Engine Corporation of China 9 languages বাংলা Deutsch فارسی Français Bahasa Indonesia 日本語 Русский Українська 中文 Edit links Article Talk English Read Edit View history Tools Tools move to sidebar hide Actions Read Edit View history General What links here Related changes Upload file Special pages Permanent link Page information Cite this page Get shortened URL Download QR code Print/export Download as PDF Printable version In other projec

In [8]:
def punch(text : str):
  return text.translate(str.maketrans('', '', string.punctuation))

In [9]:
def url_rem(text : str):
  return re.sub(r"http\S+", "", text)

In [10]:
df["E"]=df["E"].apply(lambda x: punch(x))
df["S"]=df["S"].apply(lambda x: punch(x))
df["G"]=df["G"].apply(lambda x: punch(x))

In [11]:
df["E"]=df["E"].apply(lambda x: url_rem(x))
df["S"]=df["S"].apply(lambda x: url_rem(x))
df["G"]=df["G"].apply(lambda x: url_rem(x))

In [12]:
df["ESG Rating"].nunique()

7

In [13]:
df.iloc[22]["E"]

'AECC Aeroengine Control Co Ltd ESG Risk Rating Google Tag Manager noscript End Google Tag Manager noscript Skip to main content About Us Careers Sign In Investor Solutions Investor Solutions ESG Research ESG Regulatory Solutions Climate Solutions Analytic  Reporting Solutions Stewardship Services Index Research Services Methodology Clarification ESG Risk Ratings Impact Solutions Compliance  ESG Screening Solutions Controversies Research Country Research  Ratings SubSovereign Risk Ratings ESG Risk Smart Score EU Sustainable Finance Action Plan Solutions EU Taxonomy Solution Sustainable Finance Disclosure Regulation European Banking Authority EBA Pillar III Solutions ESG Regulatory Insights Hub Climate Data and Research Low Carbon Transition Ratings Physical Climate Risk Metrics Carbon Emissions Data TCFD Disclosure Global Access Managed Screening Service API  Data Feed Solutions Partner Platforms Morningstar Sustainability Rating for Funds Engagement Services ESG Voting Policy Overlay 

In [14]:
df["allText"]=df["E"]+df["S"]+df["G"]

In [15]:
df.drop(["E","S","G"],axis=1,inplace=True)

In [16]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
label_encoder = LabelEncoder()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [17]:
df['label'] = label_encoder.fit_transform(df['ESG Rating'])

In [18]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [19]:
class ESGDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        # Tokenize text
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt"
        )

        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "label": torch.tensor(label, dtype=torch.long)
        }

In [20]:
train_dataset = ESGDataset(train_df['allText'].tolist(), train_df['label'].tolist(), tokenizer)
val_dataset = ESGDataset(val_df['allText'].tolist(), val_df['label'].tolist(), tokenizer)

In [21]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [22]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased",num_labels=len(df['label'].unique()))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")#"cuda" if torch.cuda.is_available() else
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [23]:
torch.cuda.empty_cache()

In [24]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)
criterion = torch.nn.CrossEntropyLoss()

In [25]:
def train_model(model, train_loader, val_loader, epochs=3):
    for epoch in range(epochs):
        model.train()
        total_train_loss = 0

        for batch in tqdm(train_loader):
            optimizer.zero_grad()

            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_train_loss += loss.item()

            loss.backward()
            optimizer.step()

        avg_train_loss = total_train_loss / len(train_loader)
        print(f"Epoch {epoch+1} | Train Loss: {avg_train_loss}")

        model.eval()
        total_val_loss = 0
        correct_predictions = 0

        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["label"].to(device)

                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                total_val_loss += loss.item()

                preds = torch.argmax(outputs.logits, dim=1)
                correct_predictions += torch.sum(preds == labels)

        avg_val_loss = total_val_loss / len(val_loader)
        accuracy = correct_predictions.double() / len(val_dataset)
        print(f"Epoch {epoch+1} | Validation Loss: {avg_val_loss} | Accuracy: {accuracy}")


In [26]:
train_model(model, train_loader, val_loader, epochs=3)

100%|██████████| 40/40 [05:05<00:00,  7.63s/it]


Epoch 1 | Train Loss: 1.934638500213623
Epoch 1 | Validation Loss: 1.870978021621704 | Accuracy: 0.21250000000000002


100%|██████████| 40/40 [05:03<00:00,  7.59s/it]


Epoch 2 | Train Loss: 1.890236097574234
Epoch 2 | Validation Loss: 1.8450025081634522 | Accuracy: 0.275


100%|██████████| 40/40 [05:01<00:00,  7.53s/it]


Epoch 3 | Train Loss: 1.8672594130039215
Epoch 3 | Validation Loss: 1.8504160284996032 | Accuracy: 0.275
